According to [NASA Extragalactic Database](https://ned.ipac.caltech.edu/byname?objname=Large+Magellanic+Cloud&hconst=67.8&omegam=0.308&omegav=0.692&wmap=4&corr_z=1) the LMC has spectral flux density of 3230 Jy in the V band (\~550 nm). I take this number to represent the SFD over the r-band, \~550 nm to \~650 nm ($5.4 \times 10^{14}$ Hz to $4.6 \times 10^{14}$ Hz) and estimate how many photons per pixel we will get in a 15 s exposure. To make things easy for myself, I take the throughput of the r-band to be 0.6 over the passband, which is roughly the value in the middle of the passband (\~600 nm) (Figure 4 in the [LSST Overview Paper](https://arxiv.org/pdf/0805.2366.pdf)).

In [1]:
import astropy.units as u
import numpy as np

In [2]:
sfd = 3230 * u.jansky
exposureTime = 15 * u.second
planckConstant = 6.626e-34 * u.joule * u.second
m1effectiveRadius = 3.2 * u.meter
m1Area = np.pi * m1effectiveRadius**2
fov = 9.6 * (u.degree)**2
angularDiameterLMC = 41510 * u.arcsec
angularDiameterLMC = angularDiameterLMC.to(u.degree)
angularAreaLMC = np.pi * (angularDiameterLMC/2)**2
nPixels = 3.2e9
throughput = 0.6
platescale = 0.2 * u.arcsec

The SFD is integrated over the solid angle of the source. The primary mirror collects light from Rubin's 9.6 sq degree fov, so the the characteristic SFD of one pointing is $\frac{9.6 \text{deg}^2}{\text{LMC Solid Angle}} * \Phi$, where $\Phi$ is the SFD. 

In [3]:
charSFD = sfd * fov / angularAreaLMC
charSFD

<Quantity 296.95005413 Jy>

The energy per frequency collected by the primary mirror in a 15 second exposure is (mirror area)\*(exposure time)\*$\Phi_{\text{char}}$

In [4]:
energyPerFreq = m1Area * exposureTime * charSFD
dimensionlessEPF = energyPerFreq / planckConstant
throughput * dimensionlessEPF.decompose()

<Quantity 1.29755064e+12>

The number of photons landing on the camera is the integral over the passband of (throughput)\*(energy per frequency)\/(energy per photon of that frequency)

$n_{\text{photons}} = 0.6 \cdot  \int_{\nu_0}^{\nu_1} \frac{\text{energy per frequency}} {h \nu} d\nu = 1.2975506 \times 10^{12} \int_{\nu_0}^{\nu_1} \frac{d\nu}{\nu}$

In [5]:
nPhotons = throughput * dimensionlessEPF.decompose() * np.log(5.4/4.6)
nPhotons

<Quantity 2.08052708e+11>

distributing these uniformly over the 3.2 billion pixels gives...

In [6]:
nPhotons / nPixels

<Quantity 65.01647129>

Using Duncan's suggestion of multiplying by the platescale^2 / (LMC solid angle) I get a very similar answer

In [7]:
photonsPerPixel = throughput * exposureTime * m1Area * (sfd/planckConstant) * np.log(5.4/4.6) * platescale**2 / angularAreaLMC
photonsPerPixel.decompose()

<Quantity 66.88937375>